In [1]:
%matplotlib inline
import numpy as np

from astropy import units as u
# from astropy.constants import c
from astropy.constants import k_B, G

In [2]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

As discussed in our previous telecon, we can use equation 10.19 from Stahler & Palla's book
\begin{equation}
\delta u_n = \delta u_i \left(1 - \frac{i\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^{-1}~,
\label{eq:1}
\end{equation}
where 
$n_i\langle \sigma_{in} u_i' \rangle$
is "the frequency with which a given natural atom or molecule is struck by ions", $\delta u_n$ is the perturbation on the neutral's velocity, and $\delta u_i$ is the perturbation on the ion's velocity.

The other relation to use is equation 10.21 from Stahler & Palla's book
\begin{equation}
\frac{\omega^2}{k^2} = \frac{B_o^2}{4\pi \rho_o} 
                       \left(1 - \frac{i\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)~,
\label{eq:2}
\end{equation}
which relates the $\omega$ and $k$.

Finally, equation 10.17 from Stahler & Palla's book relates the perturbation in the magnetic field as
\begin{equation}
\delta u_i = -\frac{\omega}{k} \frac{\delta B}{B_o}~.
\label{eq:3}
\end{equation}

We estimate the velocity perturvations as the velocity dispersions, $\delta u =\sqrt{2}\, \sigma_{\rm NT}$, derived from the spectral lines and therefore rewrite equation \ref{eq:1} as 
\begin{equation}
\left| \frac{\delta u_i}{\delta u_n} \right|^2 =
1 + \left(\frac{\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^{2}
\approx \left(\frac{\sigma_{\rm NT}({\rm N_2H^+})}{\sigma_{\rm NT}({\rm NH_3})}\right)^2~.
\label{eq:4}
\end{equation}

Now, equation \ref{eq:2} can also be rewritten as
\begin{equation}
\left|\frac{\omega^2}{k^2}\right| = \left(\frac{B_o^2}{4\pi \rho_o}\right)
                       \left[1 + \left(\frac{\omega}{n_i\langle \sigma_{in} u_i' \rangle}\right)^2\right]^{1/2}
\approx \frac{B_o^2}{4\pi \rho_o}
\frac{\sigma_{\rm NT}({\rm N_2H^+})}{\sigma_{\rm NT}({\rm NH_3})}~.
\label{eq:5}
\end{equation}

Combining equations \ref{eq:3}, \ref{eq:4} and \ref{eq:5} we obtain
$$
\frac{\delta B}{B_o} = \sqrt{\sigma_{\rm NT}({\rm N_2H^+}) \sigma_{\rm NT}({\rm NH_3})}
\frac{\sqrt{8\pi \rho_o}}{B_o}
$$
or
$$
\delta B= \sqrt{8\pi \rho_o} \sqrt{\sigma_{\rm NT}({\rm N_2H^+}) \sigma_{\rm NT}({\rm NH_3})}
$$

The term $n_i$ is estimated using the ionization degree from Caselli et al. (2002)
$$
x(e) = \frac{n_i}{n({\rm H_2})} = 5.2\times 10^{-6} \left( \frac{n({\rm H_2})}{{\rm cm}^{-3}}\right)^{-0.56} 
$$
and the term
$$
\langle \sigma_{in} u_i' \rangle = 1.69\times 10^{-9} {\rm cm^3 s^{-1}}
$$
is approximated by the Langevin term for HCO$^+$-H$_2$ collisions (McDaniels & Mason, 1973).

Using the relation 
$$
\lambda = \frac{2\pi}{k} = \frac{2\pi}{\omega} \frac{\omega}{k}
$$ 
and that 
$$
\frac{1}{\omega} = \frac{1}{n_i \langle \sigma_{in} u_i' \rangle}
\sqrt{\frac{\sigma_v({\rm NH_3})^2}{\sigma_v({\rm N_2H^+})^2-\sigma_v({\rm NH_3})^2}}
$$
then we can write
\begin{equation}
\lambda = \sqrt{\frac{\pi}{\rho_o}}  \frac{B_o}{n_i \langle \sigma_{in} u_i' \rangle} 
\sqrt{\frac{\sigma_v({\rm NH_3}) \sigma_v({\rm N_2H^+})}{\sigma_v({\rm N_2H^+})^2 - \sigma_v({\rm NH_3})^2}}
\end{equation}

In [85]:
def sigma_thermal(mu_mol, tk=10*u.K):
    """
    Returns the sound speed for temperature Tk and molecular weight mu.
    This is also used to determine the thermal velocity dispersion of
    a molecular transition.

    """
    return np.sqrt(k_B * tk/(mu_mol * u.u)).to(u.km/u.s)


def density_ion(dens_all, do_Caselli=True):
    """
    1.3e-5 x n(H2)^{0.5}  (from McKee 1989) or 5.2e-6 x n*H2)^{0.44} 
    """
    if do_Caselli:
        xe = 5.2e-6 * (dens_all/(u.cm**-3))**-0.56
    else:
        xe = 1.3e-5 * (dens_all/(u.cm**-3))**-0.5
    return xe*dens_all


def get_omega(sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, density=1e6/u.cm**3, do_Caselli=True):
    """
    """
    n_i = density_ion(density, do_Caselli=do_Caselli)
    return (sig_in_v_i * n_i * np.sqrt((sigma_ion/sigma_neutral)**2 - 1)).decompose()


def get_wavelength(Bfield=100*u.uG, sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, 
                   n_H2=1e6/u.cm**3, do_Caselli=True):
    """
    """
    rho0 = (n_H2 * u.u * 2.8).cgs
    n_i = density_ion(n_H2, do_Caselli=do_Caselli).cgs
    my_wave = np.sqrt(np.pi/rho0) * (Bfield.to(u.G)/(n_i*sig_in_v_i)) * np.sqrt(sigma_ion.cgs*sigma_neutral.cgs/(sigma_ion.cgs**2 - sigma_neutral.cgs**2))
    return (my_wave.value) * u.cm


def alven_speed(Bfield=100*u.G, n_H2=1e6/u.cm**3):
    """
    """
    rho_o = (n_H2 * u.u * 2.8).cgs
    V_a = Bfield.to(u.G)/np.sqrt(4*np.pi*rho_o)
    return (V_a.value)*(u.cm/u.s)


def delta_B(Bfield=100*u.uG, sigma_ion=0.1*u.km/u.s, sigma_neutral=0.08*u.km/u.s, 
                   n_H2=1e6/u.cm**3):
    rho_o = (n_H2 * u.u * 2.8).cgs
    delta_B_Bo = np.sqrt(sigma_ion * sigma_neutral) * np.sqrt(8 * np.pi * rho_o) / Bfield.to((u.g/u.cm)**(1/2)/u.s, equivalencies=equiv_B)
    return delta_B_Bo.decompose()


sig_in_v_i = 1.69e-9*u.cm**3/u.s
gauss_B = (u.g/u.cm)**(0.5)/u.s
gauss2_B = (u.g/u.cm)/u.s**2
equiv_B = [(u.G, gauss_B, lambda x: x, lambda x: x)]
equiv_B2 = [(u.G**2, gauss2_B, lambda x: x, lambda x: x)]

B_o = np.array([500.]) * u.uG
c_sound = sigma_thermal(2.37, tk=9.7*u.K).cgs
# sigma_ion = 0.59 * c_sound
# sigma_neutral = 0.48 * c_sound   
# <sigma_nt,NH3> = 0.0941 km / s
# <sigma_nt,N2H+> = 0.113 km / s
# sigma_ion = 0.113 * u.km / u.s
sigma_ion = 0.109 * u.km / u.s
# sigma_neutral = 0.094 * u.km / u.s
sigma_neutral = 0.088 * u.km / u.s
n_H2 = 7e4 / u.cm**3
rho_o = (n_H2 * u.u * 2.8).cgs
n_H2_out = 5e3 / u.cm**3
rho_o_out = (n_H2_out * u.u * 2.8).cgs

In [86]:
delta_B_Bo = np.round(delta_B(Bfield=B_o, sigma_ion=sigma_ion, n_H2=n_H2), decimals=2)
print(delta_B_Bo)

[0.05]


In [87]:
delta_B = np.round(delta_B_Bo * B_o)
print(delta_B)

[25.] uG


This corresponds to a variation of $27\,\mu$G, which corresponds at least 5\% of the field ($\delta B/B_o$).

If we want to estimate the wavelength then we will need to estimate 
$n_i\langle \sigma_{in} u_i' \rangle$ to derive $\omega$ and then obtain $k$.

We use the velocity dispersions for ions and neutrals, the mean density of the coherent core, and the magnetic field strengths.

In [88]:
Omega_MHD = get_omega(sigma_ion=sigma_ion, sigma_neutral=sigma_neutral, density=n_H2, do_Caselli=True)

In [89]:
np.round((get_wavelength(Bfield=B_o, sigma_ion=sigma_ion, sigma_neutral=sigma_neutral, n_H2=n_H2)).to(u.pc), decimals=2)

<Quantity [0.64] pc>

Therefore, the wavelength is abount 0.59 pc, where the uncertainty is dominated by the magnetic field strength. The size of the coherent region is 0.34 pc (in diameter).

In [90]:
(1/Omega_MHD).to(u.kyr)

<Quantity 36.41688199 kyr>

In [91]:
Omega_0 = (sig_in_v_i*density_ion(n_H2, do_Caselli=True))

In [92]:
np.round((1/Omega_0).to(u.kyr))

<Quantity 27. kyr>

In [93]:
(sigma_ion/c_sound).decompose()

<Quantity 0.59087785>

In [94]:
(sigma_neutral/c_sound).decompose()

<Quantity 0.477039>

In [95]:
(0.18447129083114888*u.km/u.s/c_sound).decompose()

<Quantity 1.>